<a href="https://colab.research.google.com/github/LeoDardanello/DA_Semantic_Segmentation/blob/main/Semantic_DA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import sys, os
import numpy as np
import torchvision

drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import subprocess

percorso_zip = "/content/drive/MyDrive/AML/Cityscapes.zip"

percorso_destinazione = "./"

comando_7zip = f"7z x '{percorso_zip}' -o'{percorso_destinazione}'"

subprocess.run(comando_7zip, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)


CompletedProcess(args="7z x '/content/drive/MyDrive/AML/Cityscapes.zip' -o'./'", returncode=2, stdout=b'\n7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21\np7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)\n\nScanning the drive for archives:\n1 file, 4973983436 bytes (4744 MiB)\n\nExtracting archive: /content/drive/MyDrive/AML/Cityscapes.zip\n--\nPath = /content/drive/MyDrive/AML/Cityscapes.zip\nType = zip\nERRORS:\nHeaders Error\nPhysical Size = 4973983436\n64-bit = +\n\n\n\nArchives with Errors: 1\n\nOpen Errors: 1\n\n', stderr=b'\nERRORS:\nHeaders Error\n\n')

In [2]:
if os.path.exists("/content/DA_Semantic_Segmentation"):
  !rm -rf "/content/DA_Semantic_Segmentation"
!git clone "https://github.com/LeoDardanello/DA_Semantic_Segmentation.git"

Cloning into 'DA_Semantic_Segmentation'...
remote: Enumerating objects: 148, done.
remote: Counting objects: 100% (148/148), done.
remote: Compressing objects: 100% (121/121), done.
remote: Total 148 (delta 80), reused 58 (delta 26), pack-reused 0
Receiving objects: 100% (148/148), 534.76 KiB | 5.00 MiB/s, done.
Resolving deltas: 100% (80/80), done.


In [ ]:
# os.chdir("DA_Semantic_Segmentation/")
# !ls
# !git pull

**TRAINING**

In [3]:
!pip install tensorboardX

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 1.3 MB/s eta 0:00:00


In [ ]:
!python3 "/content/DA_Semantic_Segmentation/train.py" --num_epochs 50 --num_workers 2 --checkpoint_step 5 --validation_step 10 --save_model_path "/content/DA_Semantic_Segmentation/out" --pretrain_path "/content/drive/MyDrive/STDCNet813M_73.91.tar"

use pretrain model /content/drive/MyDrive/STDCNet813M_73.91.tar
Traceback (most recent call last):
  File "/content/DA_Semantic_Segmentation/train.py", line 264, in <module>
    main()
  File "/content/DA_Semantic_Segmentation/train.py", line 241, in main
    model = BiSeNet(backbone=args.backbone, n_classes=n_classes, pretrain_model=args.pretrain_path, use_conv_last=args.use_conv_last, training_model=args.training_path)
  File "/content/DA_Semantic_Segmentation/model/model_stages.py", line 246, in __init__
    self.cp = ContextPath(backbone, pretrain_model,training_model=training_model, use_conv_last=use_conv_last)
  File "/content/DA_Semantic_Segmentation/model/model_stages.py", line 104, in __init__
    self.backbone = STDCNet813(pretrain_model=pretrain_model, use_conv_last=use_conv_last, training_model=training_model)
  File "/content/DA_Semantic_Segmentation/model/stdcnet.py", line 149, in __init__
    self.init_weight(pretrain_model)
  File "/content/DA_Semantic_Segmentation/mode

# GTA 5 TEST


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from torchvision.transforms import ToPILImage


transform=ToPILImage()
image,label=GATTO[0]
image=transform(image)


# if not torch.all(label>=0 & label<19 | label==255):
#   print("ERRORE")

plt.imshow(np.array(image))
plt.show()
plt.imshow(label.numpy(),cmap="gray")
plt.show()

In [4]:
import subprocess

percorso_zip = "/content/drive/MyDrive/AML/GTA5.zip"

percorso_destinazione = "./"

comando_7zip = f"7z x '{percorso_zip}' -o'{percorso_destinazione}'"

subprocess.run(comando_7zip, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

CompletedProcess(args="7z x '/content/drive/MyDrive/AML/GTA5.zip' -o'./'", returncode=2, stdout=b'\n7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21\np7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)\n\nScanning the drive for archives:\n1 file, 6215145033 bytes (5928 MiB)\n\nExtracting archive: /content/drive/MyDrive/AML/GTA5.zip\n--\nPath = /content/drive/MyDrive/AML/GTA5.zip\nType = zip\nERRORS:\nHeaders Error\nPhysical Size = 6215145033\n64-bit = +\n\n\n\nArchives with Errors: 1\n\nOpen Errors: 1\n\n', stderr=b'\nERRORS:\nHeaders Error\n\n')

In [ ]:
!python3 "/content/DA_Semantic_Segmentation/train.py" --num_epochs 50 --num_workers 2 --dataset_train "GTA5" --checkpoint_step 5 --validation_step 10 --save_model_path "/content/DA_Semantic_Segmentation/out" --pretrain_path "/content/drive/MyDrive/STDCNet813M_73.91.tar"

use pretrain model /content/drive/MyDrive/STDCNet813M_73.91.tar
epoch 0, lr 0.010000: 100% 1250/1250 [02:06<00:00,  9.92it/s, loss=2.565052]
loss for train : 2.976872
epoch 1, lr 0.009820: 100% 1250/1250 [02:06<00:00,  9.91it/s, loss=2.086204]
loss for train : 2.375303
epoch 2, lr 0.009639: 100% 1250/1250 [02:03<00:00, 10.08it/s, loss=1.928802]
loss for train : 2.155220
epoch 3, lr 0.009458: 100% 1250/1250 [02:00<00:00, 10.35it/s, loss=2.080985]
loss for train : 2.294462
epoch 4, lr 0.009277: 100% 1250/1250 [02:02<00:00, 10.24it/s, loss=1.929485]
loss for train : 2.143681
epoch 5, lr 0.009095: 100% 1250/1250 [02:01<00:00, 10.30it/s, loss=1.775202]
loss for train : 2.018796
epoch 6, lr 0.008913:  30% 380/1250 [00:37<01:46,  8.20it/s, loss=5.853404]

In [10]:
!rm -rf "/content/GTA5/TrainID"